#### Collab Related Only

- Connecting to Google Drive;
- Installing packages in the current kernel;

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')
root = '/content/gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
# Installing missing pip packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install magenta

##### CUDA. Optional.
If Cuda drivers are missing, run the next cell.

In [ ]:
# Check libcudnn8 version
!apt-cache policy libcudnn8
# Install latest version
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

# Export env variables
!export PATH=/usr/local/cuda-11.4/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-11.4/lib64:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-11.4/include:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64

#### Create NoteSequences
Convert a collection of MIDI files into NoteSequences. Requirement: have all midi files already uploaded to Colab.

In [ ]:
# Move uploaded files into data folder
data_path = '/content/dataset'
!find '/content/' -maxdepth 1 -type f -print0 | xargs -0 mv -t $data_path

# TFRecord file that will contain NoteSequence protocol buffers
tf_record_path = '/content/TFRecord/sequences.tfr'

!python /usr/local/lib/python3.7/dist-packages/magenta/scripts/convert_dir_to_note_sequences.py --input_dir=$data_path --output_file=$tf_record_path --recursive

#### Create SequenceExamples
SequenceExamples are fed into the model during training and evaluation. Each SequenceExample will contain a sequence of inputs and a sequence of labels that represent a melody.

In [ ]:
seq_examples_path = '/content/seq_examples'
# Available configs: basic, mono, lookback, attention
config = 'attention_rnn'

!python /usr/local/lib/python3.7/dist-packages/magenta/models/melody_rnn/melody_rnn_create_dataset.py --config=$config --input=$tf_record_path --output_dir=$seq_examples_path --eval_ratio=0.0

#### Train the Model


In [ ]:
!mkdir checkpoints
checkpoints_path = '/content/checkpoints'
example_file = seq_examples_path + '/training_melodies.tfrecord'
hparams = 'batch_size=128,rnn_layer_sizes=[256,256]'
max_steps = 20000 

!python /usr/local/lib/python3.7/dist-packages/magenta/models/melody_rnn/melody_rnn_train.py --config=$config --run_dir=$checkpoints_path --sequence_example_file=$example_file --hparams=$hparams --num_training_steps=$max_steps

#### Generate Melodies

In [ ]:
# Set the arguments
config = 'attention_rnn'
output_path = '/content/output'
primer_midi_path = '/content/input/test.mid'
checkpoints_path = '/content/checkpoints'
hparams = 'batch_size=128,rnn_layer_sizes=[256,256]'
steps = 512

# Generate with checkpoints in Colab
!python /usr/local/lib/python3.7/dist-packages/magenta/models/melody_rnn/melody_rnn_generate.py --config=$config --run_dir=$checkpoints_path --output_dir=$output_path --num_outputs=5 --num_steps=$steps --hparams=$hparams --primer_midi=$primer_midi_path

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr